In [ ]:
!pip install spacy


In [ ]:
!python -m spacy download en_core_web_sm


2023-09-21 16:29:58.578167: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
python -m spacy init config base_config.cfg --lang en --pipeline ner
python -m spacy train base_config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id 0


2023-09-21 16:30:17.957361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
2023-09-21 16:30:29.669567: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_

In [ ]:
import spacy
import pandas as pd

# Load spaCy model with NER component
nlp = spacy.load("en_core_web_sm")

# Read your dataset (assuming it's in a CSV file with one sentence per line)
dataset_file = "redaction_train_set.csv"
with open(dataset_file, "r") as file:
    sentences = file.readlines()

# Initialize a list to store annotated data
annotated_data = []

# Process and annotate each sentence
for sentence in sentences:
    sentence = sentence.strip()  # Remove leading/trailing whitespace
    doc = nlp(sentence)

    # Use displacy to visualize and annotate named entities
    spacy.displacy.render(doc, style="ent", jupyter=True)

    # Manually annotate the entities using the visualizer.
    # You can select and label the entities in the visualized sentence.

    # After annotating, ask the user to input the entities in the format:
    # [(start_char, end_char, label), ...]
    annotations = input("Enter the entities (start, end, label) or 'q' to quit: ")

    if annotations.lower() == 'q':
        break

    # Parse the entered annotations
    entities = [eval(annotation) for annotation in annotations.split(",")]

    # Create a dictionary with "entities" key for annotation
    annotations_dict = {"entities": entities}

    # Update the Doc with the manually annotated entities
    doc.ents = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            doc.ents = list(doc.ents) + [span]

    # Append the annotated sentence to the list
    annotated_data.append((sentence, annotations_dict))

# Save the annotations to a JSONL file
output_file = "annotated_data.jsonl"
with open(output_file, "w") as file:
    for sentence, annotations in annotated_data:
        record = {
            "text": sentence,
            "annotations": annotations
        }
        file.write(json.dumps(record) + "\n")


Enter the entities (start, end, label) or 'q' to quit: 14,26,PERSON


NameError: ignored

In [ ]:
import spacy
import csv

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Read your dataset, assuming it's in a CSV file with one sentence per row
with open('redaction_train_set.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        sentence = row[0]  # Assuming the sentence is in the first column

        # Process the sentence with spaCy
        doc = nlp(sentence)

        # Extract named entities and their labels
        entities = [(ent.text, ent.label_) for ent in doc.ents]

        # Print or store the recognized entities and labels for each sentence
        print(entities)


[('Emma Thompson', 'PERSON')]
[('Alex Johnson', 'PERSON'), ('123', 'CARDINAL'), ('Apple Street', 'ORG'), ('New York City', 'GPE'), ('10001', 'DATE')]
[('Chris Evans', 'PERSON'), ('569741285', 'DATE')]
[('morning', 'TIME'), ('Emily Roberts', 'PERSON'), ('January next year', 'DATE')]
[('James Davis', 'PERSON'), ('90210', 'DATE')]
[('Sophia Adams', 'PERSON'), ('June 15th, 1990', 'DATE')]
[('afternoon', 'TIME'), ('Olivia Wilson', 'PERSON'), ('555-123-4567', 'CARDINAL'), ('555', 'CARDINAL')]
[('Liam Martinez', 'PERSON'), ('July 5th', 'DATE'), ('0612', 'DATE')]
[('Mia Wilson', 'PERSON')]
[('Noah Jackson', 'PERSON')]
[('afternoon', 'TIME'), ('Ava Anderson', 'PERSON'), ('avaanderson20', 'LANGUAGE')]
[('Ethan Murphy', 'PERSON'), ('555', 'CARDINAL')]
[('Isabella Carter', 'PERSON'), ('9876543210987654', 'DATE')]
[('Benjamin Davis', 'PERSON'), ('321', 'CARDINAL'), ('Los Angeles', 'GPE')]
[('Mia Johnson', 'PERSON'), ('Samsung Galaxy', 'ORG'), ('IMEI', 'PRODUCT'), ('123456789012345', 'DATE')]
[('mor

In [ ]:
import spacy
import csv
import json

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize a list to store training examples
training_data = []

# Read your dataset, assuming it's in a CSV file with one sentence per row
with open('redaction_train_set.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        sentence = row[0]  # Assuming the sentence is in the first column

        # Process the sentence with spaCy
        doc = nlp(sentence)

        # Extract named entities and their labels
        entities = []
        for ent in doc.ents:
            entities.append((ent.start_char, ent.end_char, ent.label_))

        # Create a training example dictionary
        example = {"text": sentence, "entities": entities}

        # Append the example to the training data
        training_data.append(example)

# Save the training data as a JSON file
with open('training_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(training_data, json_file, ensure_ascii=False)

# Now, 'training_data.json' is in the desired format for fine-tuning the spaCy model


In [ ]:
import spacy
import random
import json
from spacy.training.example import Example

# Create a blank spaCy model
nlp = spacy.blank("en")

# Define and add the NER component to the pipeline
ner = nlp.add_pipe("ner")

# Load your training data in JSON format
with open('training_data.json', 'r', encoding='utf-8') as json_file:
    training_data = json.load(json_file)

# Define function to convert JSON format data to spaCy Example
def convert_to_example(data):
    text = data["text"]
    entities = data["entities"]
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, {"entities": entities})
    return example

# Convert JSON format training data to spaCy Examples
examples = [convert_to_example(data) for data in training_data]

# Initialize random weights for NER component
ner.initialize(lambda: examples)

# Fine-tune the NER model using the annotated data
for epoch in range(30):  # Adjust the number of epochs as needed
    random.shuffle(examples)
    losses = {}

    for batch in spacy.util.minibatch(examples, size=2):
        nlp.update(batch, drop=0.5, losses=losses)

    print(f"Epoch {epoch+1}, Losses: {losses}")

# Save the fine-tuned NER model
nlp.to_disk("fine_tuned_ner_model")


ValueError: ignored

In [ ]:
import spacy
import csv
import json
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the Matcher
matcher = Matcher(nlp.vocab)

# Define patterns for matching names, numbers, addresses, and passwords
name_pattern = [{"POS": {"IN": ["PROPN"]}}, {"POS": {"IN": ["PROPN"]}}]
number_pattern = [{"IS_DIGIT": True}]
address_pattern = [
    {"LIKE_NUM": True, "OP": "+"},  # Match one or more numeric tokens (house numbers)
    {"IS_ALPHA": True, "OP": "+"},  # Match one or more alphabetic tokens (street names)
    {"IS_SPACE": True, "OP": "?"},  # Match zero or one space token (optional)
    {"LIKE_NUM": True, "OP": "?"},  # Match zero or one numeric token (ZIP codes or additional info)
]

password_pattern = [{"IS_ALPHA": True, "OP": "+"}, {"IS_DIGIT": True, "OP": "+"}, {"LENGTH": {">=": 8}}]

# Add the patterns to the Matcher with custom labels
matcher.add("NAME", [name_pattern])
matcher.add("NUMBER", [number_pattern])
matcher.add("ADDRESS", [address_pattern])
matcher.add("PASSWORD", [password_pattern])


training_data = []

# Read your dataset, assuming it's in a CSV file with one row and no headers
with open('redaction_train_set.csv', 'r') as csv_file:
    # Read the single row from the CSV file
    reader = csv.reader(csv_file)
    row = next(reader)

    # Combine the sentences into one text
    text = ' '.join(row)

    # Process the text with spaCy
    doc = nlp(text)

    # Get matches and their spans
    matches = matcher(doc)
    entities = []
    for match_id, start, end in matches:
        span = doc[start:end]
        label = nlp.vocab.strings[match_id]  # Get the label from the Matcher
        entities.append({"start": start, "end": end, "label": label})

    # Create a training example dictionary
    example = {"text": text, "entities": entities}

    # Append the example to the training data list
    training_data.append(example)

# Save the training data as a JSON file in the required format for spaCy training
with open('training_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(training_data, json_file, ensure_ascii=False)

In [ ]:
import spacy
import random
import json

# Load a blank spaCy model with only the NER component
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Load the training data from the JSON file
with open("/content/training_data.json", "r", encoding="utf-8") as json_file:
    training_data = json.load(json_file)

# Add labels for NER
for _, annotations in training_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent["label"])

# Initialize the training loop
optimizer = nlp.begin_training()

# Training data in the format (text, {"entities": [(start, end, label), ...]})
for _ in range(30):  # You can adjust the number of epochs
    random.shuffle(training_data)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(training_data, size=2):
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, drop=0.5, losses=losses)

    print("Epoch {}, Losses: {}".format(_, losses))

# Save the trained model
nlp.to_disk("custom_ner_model")

# Load the trained model for later use
# custom_nlp = spacy.load("custom_ner_model")

# Test the model on new text
# doc = custom_nlp("Your test sentence goes here.")
# for ent in doc.ents:
#     print(ent.text, ent.label_)


AttributeError: ignored

In [ ]:
import spacy
import random
import json

# Load a blank spaCy model with only the NER component
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Load the training data from the JSON file
with open("/content/training_data.json", "r", encoding="utf-8") as json_file:
    training_data = json.load(json_file)

# Add labels for NER
for example in training_data:
    text = example["content"]
    entities = example["entities"]
    for ent in entities:
        ner.add_label(ent["label"])

# Initialize the training loop
optimizer = nlp.begin_training()

# Training data in the format (text, {"entities": [(start, end, label), ...]})
for _ in range(30):  # You can adjust the number of epochs
    random.shuffle(training_data)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(training_data, size=2):
        texts = [example["content"] for example in batch]
        annotations = [example["entities"] for example in batch]
        nlp.update(texts, annotations, drop=0.5, losses=losses)

    print("Epoch {}, Losses: {}".format(_, losses))

# Save the trained model
nlp.to_disk("custom_ner_model")


ValueError: ignored

In [ ]:
import json
import spacy
import random
from spacy.training.example import Example

# Create a blank spaCy model
nlp = spacy.blank("en")

# Define and add the NER component to the pipeline
ner = nlp.add_pipe("ner")

# Load your training data in JSON format
with open('training_data.json', 'r', encoding='utf-8') as json_file:
    training_data = json.load(json_file)

# Convert JSON format training data to spaCy Examples
examples = []
for data in training_data:
    text = data["text"]
    entities = data["entities"]
    example = Example.from_dict(nlp.make_doc(text), {"entities": entities})
    examples.append(example)

# Initialize random weights for NER component
ner.initialize(lambda: examples)

# Fine-tune the NER model using the annotated data
for epoch in range(30):  # Adjust the number of epochs as needed
    random.shuffle(examples)
    losses = {}

    for batch in spacy.util.minibatch(examples, size=2):
        nlp.update(batch, drop=0.5, losses=losses)

    print(f"Epoch {epoch+1}, Losses: {losses}")

# Save the fine-tuned NER model
nlp.to_disk("fine_tuned_ner_model")


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Hi, my name is Emma Thompson, and I have forgotten..." with entities "[[5, 7, 'NAME']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Epoch 1, Losses: {'ner': 11.5}
Epoch 2, Losses: {'ner': 9.644946932792664}
Epoch 3, Losses: {'ner': 7.815921425819397}
Epoch 4, Losses: {'ner': 6.534358471632004}
Epoch 5, Losses: {'ner': 5.215380027890205}
Epoch 6, Losses: {'ner': 4.235481485724449}
Epoch 7, Losses: {'ner': 3.1707327142357826}
Epoch 8, Losses: {'ner': 2.152984391897917}
Epoch 9, Losses: {'ner': 1.3347723241895437}
Epoch 10, Losses: {'ner': 1.2697494034655392}
Epoch 11, Losses: {'ner': 0.694668014999479}
Epoch 12, Losses: {'ner': 0.4506856445223093}
Epoch 13, Losses: {'ner': 0.42553007462993264}
Epoch 14, Losses: {'ner': 0.11096235444711056}
Epoch 15, Losses: {'ner': 0.11830819259193959}
Epoch 16, Losses: {'ner': 0.17797493515718088}
Epoch 17, Losses: {'ner': 0.08254448256775504}
Epoch 18, Losses: {'ner': 0.01038091994610113}
Epoch 19, Losses: {'ner': 0.0032358982315372486}
Epoch 20, Losses: {'ner': 0.0048476604875205}
Epoch 21, Losses: {'ner': 0.0003913737293457231}
Epoch 22, Losses: {'ner': 0.009173259779036158}
Epoc

In [ ]:
import spacy
import csv
import json
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the Matcher
matcher = Matcher(nlp.vocab)

# Define patterns for matching names, numbers, addresses, and passwords
name_pattern = [{"POS": {"IN": ["PROPN"]}}, {"POS": {"IN": ["PROPN"]}}]
number_pattern = [{"IS_DIGIT": True}]
address_pattern = [
    {"LIKE_NUM": True, "OP": "+"},  # Match one or more numeric tokens (house numbers)
    {"IS_ALPHA": True, "OP": "+"},  # Match one or more alphabetic tokens (street names)
    {"IS_SPACE": True, "OP": "?"},  # Match zero or one space token (optional)
    {"LIKE_NUM": True, "OP": "?"},  # Match zero or one numeric token (ZIP codes or additional info)
]

password_pattern = [{"IS_ALPHA": True, "OP": "+"}, {"IS_DIGIT": True, "OP": "+"}, {"LENGTH": {">=": 8}}]

# Add the patterns to the Matcher with custom labels
matcher.add("NAME", [name_pattern])
matcher.add("NUMBER", [number_pattern])
matcher.add("ADDRESS", [address_pattern])
matcher.add("PASSWORD", [password_pattern])

training_data = []

# Read your dataset, assuming it's in a CSV file with one row and no headers
with open('redaction_train_set.csv', 'r') as csv_file:
    # Read the single row from the CSV file
    reader = csv.reader(csv_file)
    row = next(reader)

    # Combine the sentences into one text
    text = ' '.join(row)

    # Process the text with spaCy
    doc = nlp(text)

    # Get matches and their spans
    matches = matcher(doc)
    entities = []
    for match_id, start, end in matches:
        span = doc[start:end]
        label = nlp.vocab.strings[match_id]  # Get the label from the Matcher
        entities.append([start, end, label])  # Format as [start, end, label]

    # Create a training example dictionary
    example = {"text": text, "entities": entities}

    # Append the example to the training data list
    training_data.append(example)

# Save the training data as a JSON file in the required format for spaCy training
with open('training_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(training_data, json_file, ensure_ascii=False)


In [ ]:
import spacy
import random
import json
from spacy.training.example import Example
from spacy.training import offsets_to_biluo_tags

# Load your spaCy model and NER component
nlp = spacy.load("en_core_web_sm")
ner = nlp.get_pipe("ner")

# Load your training data in JSON format
with open('training_data.json', 'r', encoding='utf-8') as json_file:
    training_data = json.load(json_file)

# Function to check and correct entity offsets
def align_entities(text, entities):
    doc = nlp.make_doc(text)
    corrected_entities = []

    tags = offsets_to_biluo_tags(doc, entities)

    if '-' in tags:
        print(f"Misaligned entities in text: {text}")
        print(f"Tags: {tags}")
        # Manually correct entity offsets as needed
        # Example: entities[0]["start"] = new_start_offset
        #          entities[0]["end"] = new_end_offset
        #          Repeat for other misaligned entities

    for entity in entities:
        corrected_entities.append({
            "start": entity["start"],
            "end": entity["end"],
            "label": entity["label"]
        })

    return corrected_entities

# Correct and convert JSON format training data to spaCy Examples
examples = []

for data in training_data:
    text = data["text"]
    entities = align_entities(text, data["entities"])
    example = Example.from_dict(nlp.make_doc(text), {"entities": entities})
    examples.append(example)

# Initialize random weights for NER component
ner.initialize(lambda: examples)

# Fine-tune the NER model using the corrected training data
for epoch in range(30):  # Adjust the number of epochs as needed
    random.shuffle(examples)
    losses = {}

    for batch in spacy.util.minibatch(examples, size=2):
        nlp.update(batch, drop=0.5, losses=losses)

    print(f"Epoch {epoch+1}, Losses: {losses}")

# Save the fine-tuned NER model
nlp.to_disk("fine_tuned_ner_model")


Misaligned entities in text: Hi, my name is Emma Thompson, and I have forgotten the password for my email account. It's emmathompson95@gmail.com.
Tags: ['O', 'O', '-', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


TypeError: ignored

In [ ]:
import spacy
import csv
import json
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the Matcher
matcher = Matcher(nlp.vocab)

# Define patterns for matching names, numbers, addresses, and passwords
name_pattern = [{"POS": {"IN": ["PROPN"]}}, {"POS": {"IN": ["PROPN"]}}]
number_pattern = [{"IS_DIGIT": True}]
address_pattern = [
    {"LIKE_NUM": True, "OP": "+"},  # Match one or more numeric tokens (house numbers)
    {"IS_ALPHA": True, "OP": "+"},  # Match one or more alphabetic tokens (street names)
    {"IS_SPACE": True, "OP": "?"},  # Match zero or one space token (optional)
    {"LIKE_NUM": True, "OP": "?"},  # Match zero or one numeric token (ZIP codes or additional info)
]

password_pattern = [{"IS_ALPHA": True, "OP": "+"}, {"IS_DIGIT": True, "OP": "+"}, {"LENGTH": {">=": 8}}]

# Add the patterns to the Matcher with custom labels
matcher.add("NAME", [name_pattern])
matcher.add("NUMBER", [number_pattern])
matcher.add("ADDRESS", [address_pattern])
matcher.add("PASSWORD", [password_pattern])

training_data = []

# Read your dataset, assuming it's in a CSV file with one row and no headers
with open('redaction_train_set.csv', 'r') as csv_file:
    # Read the single row from the CSV file
    reader = csv.reader(csv_file)
    row = next(reader)

    # Combine the sentences into one text
    text = ' '.join(row)

    # Process the text with spaCy
    doc = nlp(text)

    # Get matches and their spans
    matches = matcher(doc)
    entities = []
    for match_id, start, end in matches:
        span = doc[start:end]
        label = nlp.vocab.strings[match_id]  # Get the label from the Matcher
        entities.append((start, end, label))  # Format as (start, end, label)

    # Create a training example dictionary
    example = {"content": text, "entities": entities}  # Use "content" instead of "text"

    # Append the example to the training data list
    training_data.append(example)

# Save the training data as a JSON file in the required format for spaCy training
with open('training_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(training_data, json_file, ensure_ascii=False)


In [ ]:
import spacy
import random
import json
from spacy.training.example import Example
from spacy.training import offsets_to_biluo_tags

# Load your spaCy model and NER component
nlp = spacy.load("en_core_web_sm")
ner = nlp.get_pipe("ner")

# Load your training data in JSON format
with open('training_data.json', 'r', encoding='utf-8') as json_file:
    training_data = json.load(json_file)

# Function to check and correct entity offsets
def align_entities(text, entities):
    doc = nlp.make_doc(text)
    corrected_entities = []

    tags = offsets_to_biluo_tags(doc, entities)

    if '-' in tags:
        print(f"Misaligned entities in text: {text}")
        print(f"Tags: {tags}")
        # Manually correct entity offsets as needed
        # Example: entities[0]["start"] = new_start_offset
        #          entities[0]["end"] = new_end_offset
        #          Repeat for other misaligned entities

    for entity in entities:
        corrected_entities.append({
            "start": entity["start"],
            "end": entity["end"],
            "label": entity["label"]
        })

    return corrected_entities

# Correct and convert JSON format training data to spaCy Examples
examples = []

for data in training_data:
    text = data["text"]
    entities = align_entities(text, data["entities"])
    example = Example.from_dict(nlp.make_doc(text), {"entities": entities})
    examples.append(example)

# Initialize random weights for NER component
ner.initialize(lambda: examples)

# Fine-tune the NER model using the corrected training data
for epoch in range(30):  # Adjust the number of epochs as needed
    random.shuffle(examples)
    losses = {}

    for batch in spacy.util.minibatch(examples, size=2):
        nlp.update(batch, drop=0.5, losses=losses)

    print(f"Epoch {epoch+1}, Losses: {losses}")

# Save the fine-tuned NER model
nlp.to_disk("fine_tuned_ner_model1")


KeyError: ignored

In [ ]:
import spacy
import random
import json
from spacy.training.example import Example

# Create a blank spaCy model
nlp = spacy.blank("en")

# Define and add the NER component to the pipeline
ner = nlp.add_pipe("ner")

# Load your training data in JSON format
with open('training_data.json', 'r', encoding='utf-8') as json_file:
    training_data = json.load(json_file)

# Convert JSON format training data to spaCy Examples
examples = []
for data in training_data:
    text = data["content"]
    entities = data["entities"]
    example = Example.from_dict(nlp.make_doc(text), {"entities": entities})
    examples.append(example)

# Initialize random weights for NER component
ner.initialize(lambda: examples)

# Fine-tune the NER model using the annotated data
for epoch in range(30):  # Adjust the number of epochs as needed
    random.shuffle(examples)
    losses = {}

    for batch in spacy.util.minibatch(examples, size=2):
        nlp.update(batch, drop=0.5, losses=losses)

    print(f"Epoch {epoch+1}, Losses: {losses}")

# Save the fine-tuned NER model
nlp.to_disk("fine_tuned_ner_model1")


Epoch 1, Losses: {'ner': 11.5}
Epoch 2, Losses: {'ner': 9.480030685663223}
Epoch 3, Losses: {'ner': 7.643797039985657}
Epoch 4, Losses: {'ner': 6.371086210012436}
Epoch 5, Losses: {'ner': 4.585914269089699}
Epoch 6, Losses: {'ner': 3.5126496627926826}
Epoch 7, Losses: {'ner': 2.771520346403122}
Epoch 8, Losses: {'ner': 2.029453467577696}
Epoch 9, Losses: {'ner': 1.1702608931809664}
Epoch 10, Losses: {'ner': 0.9778577508404851}
Epoch 11, Losses: {'ner': 0.6007604662445374}
Epoch 12, Losses: {'ner': 0.3207415898796171}
Epoch 13, Losses: {'ner': 0.15849372651427984}
Epoch 14, Losses: {'ner': 0.17070488262106664}
Epoch 15, Losses: {'ner': 0.08131436250369006}
Epoch 16, Losses: {'ner': 0.0244478685381182}
Epoch 17, Losses: {'ner': 0.022496627872442332}
Epoch 18, Losses: {'ner': 0.004896798854133522}
Epoch 19, Losses: {'ner': 0.006713592645041899}
Epoch 20, Losses: {'ner': 0.0027069021497823087}
Epoch 21, Losses: {'ner': 0.007122154161097072}
Epoch 22, Losses: {'ner': 0.0003775386222590882}
